# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

# 忽略警告訊息
import warnings
warnings.filterwarnings( 'ignore' )


data_path = '/Users/YenLin/Desktop/ML_100/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

### Training Data

In [2]:
df_train.head( 10 )

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
train_Y = df_train[ 'Survived' ]                                                # 取出  df_train 中的 'Survived' 欄位（實際已知的結果！！）
df_train = df_train.drop( [ 'PassengerId', 'Survived' ] , axis = 1 )  # 將 df_train 中的 'PassengerId' 及 'Survived' 兩欄移除

### Testing Data

In [4]:
df_test.head( 10 )

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [5]:
ids = df_test[ 'PassengerId' ]                                   # 單獨取出  df_test 中的 'PassengerId' 欄位
df_test = df_test.drop( [ 'PassengerId' ] , axis = 1 )     # 將 df_test 中的 'PassengerId' 欄位移除

### 串接 df_train 及 df_test （ = df ）

In [6]:
df = pd.concat( [ df_train, df_test ] )
df.head( 10 )

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


### 篩選出類別型態的欄位

In [7]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = [ ]
for dtype, feature in zip( df.dtypes, df.columns ) :
    if dtype == 'object' :
        object_features.append( feature )
print( f'{len(object_features)} Numeric Features : {object_features}\n' )

# 只留類別型欄位
df = df[ object_features ]

# 將遺漏值補上 'None'
df = df.fillna( 'None' )

print( f'shape of df : {df.shape}' )
df.head( ) 

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

shape of df : (1309, 5)


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


#### 標籤編碼 + 邏輯斯迴歸模型

In [8]:
df_temp = pd.DataFrame( )

# 標籤編碼
for c in df.columns :
    df_temp[ c ] = LabelEncoder( ).fit_transform( df[ c ] )

# 標籤編碼後
df_temp.head( 10 )

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3
5,818,1,373,185,2
6,767,1,109,163,3
7,914,1,541,185,3
8,605,0,477,185,3
9,847,0,174,185,0


In [9]:
# 將 'Name' 欄位移除
df_temp.drop( [ 'Name' ] , axis = 1 )

# 重新切割 training data 及 testing data 
train_num = train_Y.shape[0]
train_X = df_temp[ : train_num ]

# 線性迴歸模型
estimator = LogisticRegression( solver = 'liblinear' )

# 評估模型
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
start = time.time()

print(f'shape of training data : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape of training data : (891, 5)
score : 0.780004837244799
time : 0.03065800666809082 sec


#### 均值編碼 + 邏輯斯迴歸模型

In [10]:
data = pd.concat( [ df[ : train_num ], train_Y ], axis = 1 )
print( f'shape of data : { data.shape }' )

data.head( 10 )

shape of data : (891, 6)


,Name,Sex,Ticket,Cabin,Embarked,Survived
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1
4,"Allen, Mr. William Henry",male,373450,None,S,0
5,"Moran, Mr. James",male,330877,None,Q,0
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,0
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1


In [11]:
# 均值編碼
for c in df.columns:
    mean_df = data.groupby( [ c ] )[ 'Survived' ].mean( ).reset_index( )
    mean_df.columns = [ c, f'{c}_mean' ]                                            # 定義均值編碼後 dataframe 的欄位名稱
    data = pd.merge( data, mean_df, on = c, how = 'left' )
    data = data.drop( [c] , axis = 1 )

data.head( 10 )

,Survived,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0,0.188908,0.0,0.299854,0.336957
1,1,1,0.742038,1.0,1.000000,0.553571
2,1,1,0.742038,1.0,0.299854,0.336957
3,1,1,0.742038,0.5,0.500000,0.336957
4,0,0,0.188908,0.0,0.299854,0.336957
5,0,0,0.188908,0.0,0.299854,0.389610
6,0,0,0.188908,0.0,0.000000,0.336957
7,0,0,0.188908,0.0,0.299854,0.336957
8,1,1,0.742038,1.0,0.299854,0.336957
9,1,1,0.742038,0.5,0.299854,0.553571


In [12]:
# 邏輯斯迴歸
data = data.drop( [ 'Survived' ] , axis = 1 )
estimator = LogisticRegression( solver = 'liblinear' )

# 評估模型
start = time.time( )
print( f'shape : { train_X.shape }' )
print( f'score : { cross_val_score(estimator, data, train_Y, cv=5).mean( ) }' )
print( f'time : { time.time( ) - start } sec')

shape : (891, 5)
score : 1.0
time : 0.03201413154602051 sec


In [13]:
data_2 = pd.concat( [ df[ : train_num ], train_Y ], axis = 1 )
print( '各欄位中的相異質數：' + '\n '+ str( data_2.nunique( ) ) )

data_2.head( 10 )

各欄位中的相異質數：
 Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
Survived      2
dtype: int64


,Name,Sex,Ticket,Cabin,Embarked,Survived
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1
4,"Allen, Mr. William Henry",male,373450,None,S,0
5,"Moran, Mr. James",male,330877,None,Q,0
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,0
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1


In [14]:
# 均值編碼
for c in df.columns:
    mean_df_2 = data_2.groupby( [ c ] )[ 'Survived' ].mean( ).reset_index( )
    mean_df_2.columns = [ c, f'{c}_mean' ]                                             # 定義均值編碼後 dataframe 的欄位名稱
    data_2 = pd.merge( data_2, mean_df_2, on = c, how = 'left' )
    data_2 = data_2.drop( [c] , axis = 1 )
    
# 均值編碼後
data_2.head(10)

,Survived,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0,0.188908,0.0,0.299854,0.336957
1,1,1,0.742038,1.0,1.000000,0.553571
2,1,1,0.742038,1.0,0.299854,0.336957
3,1,1,0.742038,0.5,0.500000,0.336957
4,0,0,0.188908,0.0,0.299854,0.336957
5,0,0,0.188908,0.0,0.299854,0.389610
6,0,0,0.188908,0.0,0.000000,0.336957
7,0,0,0.188908,0.0,0.299854,0.336957
8,1,1,0.742038,1.0,0.299854,0.336957
9,1,1,0.742038,0.5,0.299854,0.553571


In [15]:
data_2 = data_2.drop( [ 'Name_mean', 'Ticket_mean', 'Survived' ] , axis = 1 )

# 邏輯斯迴歸
estimator_2 = LogisticRegression( solver = 'liblinear' )

# 評估模型
start = time.time( )
print( f'shape : { train_X.shape }')
print( f'score : { cross_val_score( estimator_2, data_2, train_Y, cv = 5 ).mean( ) }' )
print( f'time : { time.time( ) - start } sec' )

shape : (891, 5)
score : 0.8350366889413987
time : 0.0295259952545166 sec


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

####  Answer :  
 均值編碼效果較好。  
 可能原因為： 'Name'、'Ticket' 及 'Cabin' 三個欄位的種類數眾多，均值編碼較合適用來描述其差異性；然而，若欄位的種類數過多時，則均值編碼容易出現過擬合( overfitting )的結果，故在模擬前，須先考慮欄位種類的數量及欄位本身對模型的意義，再做餵入模型預測。